In [1]:
import os,re
from pyspark.sql.functions import col, array_contains, array_position,year,to_date

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1653556795446_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Importing DataFrame
df = spark.read.parquet('s3://pas-biogen-sagemaker/Antony/Ongoing_Output/Result_Par_Final.parquet')
print("number of rows: " + str(df.count()), "   ", "https://en.wikipedia.org/wiki/Muhakamat_al-Lughatayn" , "   ", sep=os.linesep)
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

number of rows: 61987184
   
https://en.wikipedia.org/wiki/Muhakamat_al-Lughatayn
   
root
 |-- patient_key: long (nullable = true)
 |-- Entity: string (nullable = true)
 |-- Entity_label: string (nullable = true)
 |-- Is_Negated: boolean (nullable = true)
 |-- UMLS_CUI: string (nullable = true)
 |-- UMLS_Name: string (nullable = true)
 |-- UMLS_child_label: string (nullable = true)
 |-- UMLS_parent_label: string (nullable = true)
 |-- UMLS_Definition: string (nullable = true)
 |-- UMLS_Span: string (nullable = true)
 |-- Test_Name: string (nullable = true)
 |-- Modifier_Result: string (nullable = true)
 |-- Numeric_Result: string (nullable = true)
 |-- SNOMED: string (nullable = true)
 |-- ICD_Code: string (nullable = true)
 |-- Thought_Disturbance: long (nullable = true)
 |-- Abnormal_Behavior: long (nullable = true)
 |-- Suicidal_Intent: long (nullable = true)
 |-- Disorientated: long (nullable = true)
 |-- Mental_Health_Impairment: long (nullable = true)
 |-- Memory_Loss: long (nul

In [3]:
# Name of columns
print(df.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['patient_key', 'Entity', 'Entity_label', 'Is_Negated', 'UMLS_CUI', 'UMLS_Name', 'UMLS_child_label', 'UMLS_parent_label', 'UMLS_Definition', 'UMLS_Span', 'Test_Name', 'Modifier_Result', 'Numeric_Result', 'SNOMED', 'ICD_Code', 'Thought_Disturbance', 'Abnormal_Behavior', 'Suicidal_Intent', 'Disorientated', 'Mental_Health_Impairment', 'Memory_Loss', 'Clouded_Consciousness', 'Agitation', 'Congnitive_Impairment', 'Anchors', 'Alzheimer', 'pat_last_name', 'pat_first_name', 'pat_mrn', 'pat_dob', 'pat_gender', 'pat_race', 'pat_ethinicity', 'pat_city', 'pat_zip', 'pat_county', 'pat_state', 'pat_country', 'pat_marital_status', 'pat_smoking_status', 'pat_status']

In [5]:

#sc.install_pypi_package("cython")
#sc.install_pypi_package("matplotlib", "https://pypi.org/simple")
sc.uninstall_package('matplotlib')
sc.install_pypi_package("matplotlib")
#sc.install_pypi_package("pandas==0.25.1")
#sc.install_pypi_package("numpy")
#sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Uninstalling matplotlib-3.1.1:
  Successfully uninstalled matplotlib-3.1.1

  Using cached https://files.pythonhosted.org/packages/df/e7/0ad4aad00d6d0314aaf97526a54a34d385f898ebb7915d112431fff452ff/matplotlib-3.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/05/8e/8de486cbd03baba4deef4142bd643a3e7bbe954a784dc1bb17142572d127/packaging-21.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/43/6e/59853546226ee6200f9ba6e574d11604b60ad0754d2cbd1c8f3246b70418/Pillow-9.1.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/2f/85/2f6e42fb4b537b9998835410578fb1973175b81691e9a82ab6668cf64b0b/fonttools-4.33.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/45/b7/de7b8e67f2232c26af57c205aaad29fe17754f793404f59c8a730c7a191a/numpy-1.21.6.zip
  Running setup.py bdist_wheel for unknown: started
  Running setup.py bdist_wheel for unknown: finished with status 'error'
  Complete outp

In [ ]:
df2 = df.limit(1000)

In [6]:
# Install Python Libraries
#import pandas as pd
#import regex as re
import matplotlib.pyplot as plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# No of unique patients
print('Unique Patients - ', df.select('patient_key').distinct().count())

In [ ]:
df_neg = df[df['Is_Negated'] == False]

In [ ]:
print('Count Is_Negated Values - ', df_neg.count() ,'/', df.count(), ' which is ', (int(df_neg.count())/int(df.count())* 100), '%')

In [ ]:
#### Calculating Age
#df_neg[['pat_dob']].show(5)

from pyspark.sql.functions import substring
df_neg = df_neg.withColumn("year", substring(col("pat_dob"), 7, 10))

In [ ]:
df_neg = df_neg.withColumn("age", (2022 - df_neg['year']))
df_neg = df_neg.withColumn("pat_age",df_neg.age.cast('int'))

df_neg[['pat_age']].show(5)

In [ ]:
#Patients having Alzheimer
df_alz = df_neg[df_neg['Alzheimer'] == 1].select(['patient_key','UMLS_CUI','UMLS_Name','pat_gender','pat_city','pat_age','Thought_Disturbance','Abnormal_Behavior','Suicidal_Intent','Disorientated','Mental_Health_Impairment','Memory_Loss','Clouded_Consciousness','Agitation','Congnitive_Impairment']).dropDuplicates()
df_overall = df_neg.select(['patient_key','UMLS_CUI','UMLS_Name','pat_gender','pat_city','pat_age','Thought_Disturbance','Abnormal_Behavior','Suicidal_Intent','Disorientated','Mental_Health_Impairment','Memory_Loss','Clouded_Consciousness','Agitation','Congnitive_Impairment']).dropDuplicates()

In [ ]:
#Patients having Anchor features Overall vs ALZ
### Thought_Disturbance
all_Thought_Disturbance = df_overall[df_overall['Thought_Disturbance'] == 1].select('patient_key')
print('Overall Thought_Disturbance', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Thought_Disturbance'] == 1].select('patient_key')
print('Alzheimer Thought_Disturbance', alz_Thought_Disturbance[['patient_key']].distinct().count())

### Abnormal_Behavior
all_Thought_Disturbance = df_overall[df_overall['Abnormal_Behavior'] == 1].select('patient_key')
print('Overall Abnormal_Behavior', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Abnormal_Behavior'] == 1].select('patient_key')
print('Alzheimer Abnormal_Behavior', alz_Thought_Disturbance[['patient_key']].distinct().count())

### Suicidal_Intent
all_Thought_Disturbance = df_overall[df_overall['Suicidal_Intent'] == 1].select('patient_key')
print('Overall Suicidal_Intent', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Suicidal_Intent'] == 1].select('patient_key')
print('Alzheimer Suicidal_Intent', alz_Thought_Disturbance[['patient_key']].distinct().count())

### Disorientated
all_Thought_Disturbance = df_overall[df_overall['Disorientated'] == 1].select('patient_key')
print('Overall Disorientated', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Disorientated'] == 1].select('patient_key')
print('Alzheimer Disorientated', alz_Thought_Disturbance[['patient_key']].distinct().count())


### Mental_Health_Impairment
all_Mental_Health_Impairment = df_overall[df_overall['Mental_Health_Impairment'] == 1].select('patient_key')
print('Overall Mental_Health_Impairment', all_Mental_Health_Impairment[['patient_key']].distinct().count())
alz_Mental_Health_Impairment = df_alz[df_alz['Mental_Health_Impairment'] == 1].select('patient_key')
print('Alzheimer Mental_Health_Impairment', alz_Mental_Health_Impairment[['patient_key']].distinct().count())


### Memory_Loss
all_Thought_Disturbance = df_overall[df_overall['Memory_Loss'] == 1].select('patient_key')
print('Overall Memory_Loss', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Memory_Loss'] == 1].select('patient_key')
print('Alzheimer Memory_Loss', alz_Thought_Disturbance[['patient_key']].distinct().count())

### Clouded_Consciousness
all_Thought_Disturbance = df_overall[df_overall['Clouded_Consciousness'] == 1].select('patient_key')
print('Overall Clouded_Consciousness', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Clouded_Consciousness'] == 1].select('patient_key')
print('Alzheimer Clouded_Consciousness', alz_Thought_Disturbance[['patient_key']].distinct().count())

### Agitation
all_Thought_Disturbance = df_overall[df_overall['Agitation'] == 1].select('patient_key')
print('Overall Agitation', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Agitation'] == 1].select('patient_key')
print('Alzheimer Agitation', alz_Thought_Disturbance[['patient_key']].distinct().count())

### Congnitive_Impairment
all_Thought_Disturbance = df_overall[df_overall['Congnitive_Impairment'] == 1].select('patient_key')
print('Overall Congnitive_Impairment', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Congnitive_Impairment'] == 1].select('patient_key')
print('Alzheimer Congnitive_Impairment', alz_Thought_Disturbance[['patient_key']].distinct().count())

In [ ]:
### Total Patients having Alzheimer
alz_cnt = df_alz[['patient_key']].distinct().count()
print('Alzheimer count', alz_cnt)

#Gender
all_Gender_M = df_overall[df_overall['pat_gender'] == 'M'].select('patient_key')
all_Gender_F = df_overall[df_overall['pat_gender'] == 'F'].select('patient_key')


alz_Gender_M = df_alz[df_alz['pat_gender'] == 'M'].select('patient_key')
alz_Gender_F = df_alz[df_alz['pat_gender'] == 'F'].select('patient_key')


print('all_Gender_M', all_Gender_M[['patient_key']].distinct().count())
print('all_Gender_F', all_Gender_F[['patient_key']].distinct().count())
print('alz_Gender_M', alz_Gender_M[['patient_key']].distinct().count())
print('alz_Gender_F', alz_Gender_F[['patient_key']].distinct().count())

###

In [ ]:
from pyspark.sql.functions import when, col
conditions = when(col("pat_age") < 50, "<50").when(col("pat_age").between(50,55), "50-55").when(col("pat_age").between(55,60), "55-60").when(col("pat_age").between(60,65), "60-65").when(col("pat_age").between(65,70), "65-70").when(col("pat_age").between(70,75), "70-75").when(col("pat_age").between(75,80), "75-80").when(col("pat_age").between(80,85), "80-85").when(col("pat_age").between(85,90), "85-90").when(col("pat_age").between(90,150), ">90")
df_alz = df_alz.withColumn("age_catg", conditions)

In [ ]:
df_age = df_alz.select(['patient_key','age_catg']).drop_duplicates()
df_age = df_age.groupBy('age_catg').count().sort("age_catg").toPandas()
df_age

In [ ]:
df_age

In [ ]:
plt.clf()
labels = [f"Age Category: {rating}" for rating in df_age['age_catg']]
reviews = [num_reviews for num_reviews in df_age['count']]
colors = ['#00876c', '#89c079', '#fff392', '#fc9e5a', '#de425b']
fig, ax = plt.subplots(figsize=(8,5))
w,a,b = ax.pie(reviews, autopct='%1.1f%%', colors=colors)
plt.title('Distribution of star ratings for books')
ax.legend(w, labels, title="Star Ratings", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))